# Artificial Intelligence Nanodegree

## Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this IPython notebook.



---
### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!). 

![Sample Dog Output](images/sample_dog_output.png)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Use a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 6](#step6): Write your Algorithm
* [Step 7](#step7): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

### Import Dog Dataset

In the code cell below, we import a dataset of dog images.  We populate a few variables through the use of the `load_files` function from the scikit-learn library:
- `train_files`, `valid_files`, `test_files` - numpy arrays containing file paths to images
- `train_targets`, `valid_targets`, `test_targets` - numpy arrays containing onehot-encoded classification labels 
- `dog_names` - list of string-valued dog breed names for translating labels

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


### Import Human Dataset

In the code cell below, we import a dataset of human images, where the file paths are stored in the numpy array `human_files`.

In [2]:
import random
random.seed(8675309)

# load filenames in shuffled human dataset
human_files = np.array(glob("lfw/*/*"))
random.shuffle(human_files)

# print statistics about the dataset
print('There are %d total human images.' % len(human_files))

There are 0 total human images.


---
<a id='step1'></a>
## Step 1: Detect Humans

We use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.

In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [3]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(human_files[3])
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x,y,w,h) in faces:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

IndexError: index 3 is out of bounds for axis 0 with size 0

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [4]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 

In [5]:
human_files_short = human_files[:100]
dog_files_short = train_files[:100]
# Do NOT modify the code above this line.

## TODO: Test the performance of the face_detector algorithm 
## on the images in human_files_short and dog_files_short.
def calc_face_detected_proportion(img_paths):
    vectorized_detector = np.vectorize(face_detector)
    return len(img_paths[vectorized_detector(img_paths)]) / len(img_paths)


print("percentage of the images hava a detected face in human_files_short: {} %".format(calc_face_detected_proportion(human_files_short) * 100) )
print("percentage of the images hava a detected face in dog_files_short: {} %".format(calc_face_detected_proportion(dog_files_short) * 100) )

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

__Question 2:__ This algorithmic choice necessitates that we communicate to the user that we accept human images only when they provide a clear view of a face (otherwise, we risk having unneccessarily frustrated users!). In your opinion, is this a reasonable expectation to pose on the user? If not, can you think of a way to detect humans in images that does not necessitate an image with a clearly presented face?

__Answer:__
I think it depends on the buisiness purpose. However, supposed to widely used api, it should not pose the limitations on user because u

With cnn, 


We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).  Please use the code cell below to design and test your own face detection algorithm.  If you decide to pursue this _optional_ task, report performance on each of the datasets.

In [6]:
## (Optional) TODO: Report the performance of another  
## face detection algorithm on the LFW dataset
### Feel free to use as many code cells as needed.

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a pre-trained [ResNet-50](http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006) model to detect dogs in images.  Our first line of code downloads the ResNet-50 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  Given an image, this pre-trained ResNet-50 model returns a prediction (derived from the available categories in ImageNet) for the object that is contained in the image.

In [7]:
from keras.applications.resnet50 import ResNet50

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

### Pre-process the Data

When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where `nb_samples` corresponds to the total number of images (or samples), and `rows`, `columns`, and `channels` correspond to the number of rows, columns, and channels for each image, respectively.  

The `path_to_tensor` function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN.  The function first loads the image and resizes it to a square image that is $224 \times 224$ pixels.  Next, the image is converted to an array, which is then resized to a 4D tensor.  In this case, since we are working with color images, each image has three channels.  Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

$$
(1, 224, 224, 3).
$$

The `paths_to_tensor` function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape 

$$
(\text{nb_samples}, 224, 224, 3).
$$

Here, `nb_samples` is the number of samples, or number of images, in the supplied array of image paths.  It is best to think of `nb_samples` as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [8]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

### Making Predictions with ResNet-50

Getting the 4D tensor ready for ResNet-50, and for any other pre-trained model in Keras, requires some additional processing.  First, the RGB image is converted to BGR by reordering the channels.  All pre-trained models have the additional normalization step that the mean pixel (expressed in RGB as $[103.939, 116.779, 123.68]$ and calculated from all pixels in all images in ImageNet) must be subtracted from every pixel in each image.  This is implemented in the imported function `preprocess_input`.  If you're curious, you can check the code for `preprocess_input` [here](https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py).

Now that we have a way to format our image for supplying to ResNet-50, we are now ready to use the model to extract the predictions.  This is accomplished with the `predict` method, which returns an array whose $i$-th entry is the model's predicted probability that the image belongs to the $i$-th ImageNet category.  This is implemented in the `ResNet50_predict_labels` function below.

By taking the argmax of the predicted probability vector, we obtain an integer corresponding to the model's predicted object class, which we can identify with an object category through the use of this [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a). 

In [9]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained ResNet-50 model, we need only check if the `ResNet50_predict_labels` function above returns a value between 151 and 268 (inclusive).

We use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [10]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

### (IMPLEMENTATION) Assess the Dog Detector

__Question 3:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ 

In [11]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
def calc_dog_detected_proportion(img_paths):
    vectorized_doc_detector = np.vectorize(dog_detector)
    num_dog_detected = len(img_paths[vectorized_doc_detector(img_paths)])
    return num_dog_detected / len(img_paths)

print("percentage of the images hava a detected dog in human_files_short: {} %".format(calc_dog_detected_proportion(human_files_short) * 100))
print("percentage of the images hava a detected dog in dog_files_short: {} %".format(calc_dog_detected_proportion(dog_files_short) * 100))

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 1%.  In Step 5 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

Be careful with adding too many trainable layers!  More parameters means longer training, which means you are more likely to need a GPU to accelerate the training process.  Thankfully, Keras provides a handy estimate of the time that each epoch is likely to take; you can extrapolate this estimate to figure out how long it will take for your algorithm to train. 

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have great difficulty in distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun! 

### Pre-process the Data

We rescale the images by dividing every pixel in every image by 255.

In [12]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:05<00:00, 169.28it/s]


### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  At the end of your code cell block, summarize the layers of your model by executing the line:
    
        model.summary()

We have imported some Python modules to get you started, but feel free to import as many modules as you need.  If you end up getting stuck, here's a hint that specifies a model that trains relatively fast on CPU and attains >1% test accuracy in 5 epochs:

![Sample CNN](images/sample_cnn.png)
           
__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  If you chose to use the hinted architecture above, describe why you think that CNN architecture should work well for the image classification task.

__Answer:__ 

In [13]:
train_tensors.shape

(6680, 224, 224, 3)

In [16]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization, SeparableConv2D
from keras.models import Model
from keras import layers

DOG_CLASSES = 133
INPUT_SHAPE = (224, 224, 3)
img_input = Input(shape=INPUT_SHAPE)

x = Conv2D(filters=64, kernel_size=3, padding='same', use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)

residual = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=128, kernel_size=3, padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=128, kernel_size=3, padding='same', use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D(3, strides=1, padding="same")(x)
x = layers.add([x, residual])


residual = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D(3, strides=1, padding="same")(x)
x = layers.add([x, residual])

x = GlobalAveragePooling2D()(x)
x = Dense(DOG_CLASSES, activation='softmax')(x)
model = Model(img_input, x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_3 (Conv2D)                (None, 224, 224, 64)  1728                                         
____________________________________________________________________________________________________
batch_normalization_5 (BatchNorm (None, 224, 224, 64)  256                                          
____________________________________________________________________________________________________
activation_52 (Activation)       (None, 224, 224, 64)  0                                            
___________________________________________________________________________________________

### Compile the Model

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [24]:
from keras.callbacks import ModelCheckpoint, History, BaseLogger
from keras.preprocessing.image import ImageDataGenerator

### TODO: specify the number of epochs that you would like to use to train the model.
RANDOM_SEED=10
epochs = 30

train_generator = ImageDataGenerator(
                            rotation_range=45,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest'
                        )


valid_generator = ImageDataGenerator(
                            rotation_range=45,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest'
                        )

train_generator.fit(train_tensors, seed=RANDOM_SEED)
valid_generator.fit(valid_tensors, seed=RANDOM_SEED)



history_storer = History()
base_logger = BaseLogger()
### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

batch_size=20
model.fit_generator(train_generator.flow(train_tensors, train_targets, batch_size=batch_size),
                    steps_per_epoch=train_tensors.shape[0],
                  validation_data=valid_generator.flow(valid_tensors, valid_targets, batch_size=batch_size),
                  epochs=epochs, validation_steps=valid_tensors.shape[0], callbacks=[checkpointer, history_storer, base_logger], verbose=1)

#history = model.fit(train_tensors, train_targets, 
#          validation_data=(valid_tensors, valid_targets),
#          epochs=epochs, batch_size=20, callbacks=[checkpointer, history_storer, base_logger], verbose=1)

Epoch 1/30
   3/6680 [..............................] - ETA: 719579s - loss: 5.2616 - acc: 0.0000e+00

KeyboardInterrupt: 

### Load the Model with the Best Validation Loss

In [ ]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the Model

Try out your model on the test dataset of dog images.  Ensure that your test accuracy is greater than 1%.

In [ ]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100 * np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1)) / len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

---
<a id='step4'></a>
## Step 4: Use a CNN to Classify Dog Breeds

To reduce training time without sacrificing accuracy, we show you how to train a CNN using transfer learning.  In the following step, you will get a chance to use transfer learning to train your own CNN.

### Obtain Bottleneck Features

In [19]:
bottleneck_features = np.load('bottleneck_features/DogVGG16Data.npz')
train_VGG16 = bottleneck_features['train']
valid_VGG16 = bottleneck_features['valid']
test_VGG16 = bottleneck_features['test']

### Model Architecture

The model uses the the pre-trained VGG-16 model as a fixed feature extractor, where the last convolutional output of VGG-16 is fed as input to our model.  We only add a global average pooling layer and a fully connected layer, where the latter contains one node for each dog category and is equipped with a softmax.

In [20]:
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dense(133, activation='softmax'))

VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 133)               68229     
Total params: 68,229.0
Trainable params: 68,229.0
Non-trainable params: 0.0
_________________________________________________________________


### Compile the Model

In [21]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Train the Model

In [22]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, train_targets, 
          validation_data=(valid_VGG16, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 3s - loss: 11.9010 - acc: 0.1355 - val_loss: 10.4682 - val_acc: 0.2263
Epoch 2/20
6680/6680 [==============================] - 3s - loss: 9.4935 - acc: 0.3105 - val_loss: 9.3726 - val_acc: 0.3269
Epoch 3/20
6680/6680 [==============================] - 3s - loss: 8.8262 - acc: 0.3813 - val_loss: 9.0196 - val_acc: 0.3521
Epoch 4/20
6680/6680 [==============================] - 3s - loss: 8.5072 - acc: 0.4169 - val_loss: 8.8755 - val_acc: 0.3665
Epoch 5/20
6680/6680 [==============================] - 3s - loss: 8.3737 - acc: 0.4415 - val_loss: 8.9443 - val_acc: 0.3689
Epoch 6/20
6680/6680 [==============================] - 3s - loss: 8.2933 - acc: 0.4564 - val_loss: 8.7117 - val_acc: 0.3916
Epoch 7/20


6680/6680 [==============================] - 3s - loss: 8.1615 - acc: 0.4672 - val_loss: 8.6916 - val_acc: 0.3880
Epoch 8/20
6680/6680 [==============================] - 3s - loss: 8.0902 - acc: 0.4799 - val_loss: 8.6082 - val_acc: 0.4024
Epoch 9/20
6680/6680 [==============================] - 3s - loss: 7.9671 - acc: 0.4913 - val_loss: 8.6032 - val_acc: 0.3940
Epoch 10/20
6680/6680 [==============================] - 3s - loss: 7.9284 - acc: 0.4967 - val_loss: 8.5281 - val_acc: 0.4036
Epoch 11/20
6680/6680 [==============================] - 3s - loss: 7.8737 - acc: 0.4985 - val_loss: 8.5043 - val_acc: 0.4168
Epoch 12/20
6680/6680 [==============================] - 3s - loss: 7.7925 - acc: 0.5054 - val_loss: 8.4654 - val_acc: 0.4096
Epoch 13/20


6680/6680 [==============================] - 3s - loss: 7.7654 - acc: 0.5100 - val_loss: 8.4743 - val_acc: 0.4036
Epoch 14/20
6680/6680 [==============================] - 3s - loss: 7.6757 - acc: 0.5108 - val_loss: 8.3696 - val_acc: 0.4168
Epoch 15/20
6680/6680 [==============================] - 3s - loss: 7.6171 - acc: 0.5190 - val_loss: 8.2946 - val_acc: 0.4287
Epoch 16/20
6680/6680 [==============================] - 3s - loss: 7.6061 - acc: 0.5217 - val_loss: 8.3678 - val_acc: 0.4180
Epoch 17/20
6680/6680 [==============================] - 3s - loss: 7.4977 - acc: 0.5237 - val_loss: 8.3458 - val_acc: 0.4180
Epoch 18/20
6680/6680 [==============================] - 3s - loss: 7.3317 - acc: 0.5295 - val_loss: 8.0856 - val_acc: 0.4335
Epoch 19/20


6680/6680 [==============================] - 3s - loss: 7.2070 - acc: 0.5388 - val_loss: 8.0765 - val_acc: 0.4299
Epoch 20/20
6680/6680 [==============================] - 3s - loss: 7.1519 - acc: 0.5442 - val_loss: 8.0193 - val_acc: 0.4407


### Load the Model with the Best Validation Loss

In [23]:
VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### Test the Model

Now, we can use the CNN to test how well it identifies breed within our test dataset of dog images.  We print the test accuracy below.

In [24]:
# get index of predicted dog breed for each image in test set
VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 46.5311%


### Predict Dog Breed with the Model

In [25]:
from extract_bottleneck_features import *

def VGG16_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = VGG16_model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

---
<a id='step5'></a>
## Step 5: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

In Step 4, we used transfer learning to create a CNN using VGG-16 bottleneck features.  In this section, you must use the bottleneck features from a different pre-trained model.  To make things easier for you, we have pre-computed the features for all of the networks that are currently available in Keras:
- [VGG-19](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogVGG19Data.npz) bottleneck features
- [ResNet-50](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogResnet50Data.npz) bottleneck features
- [Inception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogInceptionV3Data.npz) bottleneck features
- [Xception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogXceptionData.npz) bottleneck features

The files are encoded as such:

    Dog{network}Data.npz
    
where `{network}`, in the above filename, can be one of `VGG19`, `Resnet50`, `InceptionV3`, or `Xception`.  Pick one of the above architectures, download the corresponding bottleneck features, and store the downloaded file in the `bottleneck_features/` folder in the repository.

### (IMPLEMENTATION) Obtain Bottleneck Features

In the code block below, extract the bottleneck features corresponding to the train, test, and validation sets by running the following:

    bottleneck_features = np.load('bottleneck_features/Dog{network}Data.npz')
    train_{network} = bottleneck_features['train']
    valid_{network} = bottleneck_features['valid']
    test_{network} = bottleneck_features['test']

# transfer model search

In [26]:
networks = ["VGG19", "Resnet50", "InceptionV3", "Xception"]

In [27]:
def with_bottleneck(network):
    ### Obtain Bottleneck Features

    bottleneck_features = np.load('bottleneck_features/Dog{}Data.npz'.format(network))
    train_VGG16 = bottleneck_features['train']
    valid_VGG16 = bottleneck_features['valid']
    test_VGG16 = bottleneck_features['test']

    VGG16_model = Sequential()
    VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
    VGG16_model.add(Dense(133, activation='softmax'))

    VGG16_model.summary()

    ### Compile the Model

    VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    ### Train the Model
    model_file = 'saved_models/weights.best.{}.hdf5'.format(network)
    checkpointer = ModelCheckpoint(filepath=model_file, 
                                   verbose=1, save_best_only=True)

    VGG16_model.fit(train_VGG16, train_targets, 
              validation_data=(valid_VGG16, valid_targets),
              epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

    ### Load the Model with the Best Validation Loss

    VGG16_model.load_weights(model_file)

    ### Test the Model

    # get index of predicted dog breed for each image in test set
    VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

    # report test accuracy
    test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
    return (test_accuracy, model_file)

In [28]:
results = [with_bottleneck(network) for network in networks]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 133)               68229     
Total params: 68,229.0
Trainable params: 68,229.0
Non-trainable params: 0.0
_________________________________________________________________
Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 3s - loss: 12.3916 - acc: 0.1268 - val_loss: 11.2037 - val_acc: 0.1880
Epoch 2/20
6680/6680 [==============================] - 3s - loss: 10.6619 - acc: 0.2638 - val_loss: 10.6058 - val_acc: 0.2575
Epoch 3/20
6680/6680 [==============================] - 3s - loss: 10.3108 - acc: 0.3130 - val_loss: 10.5507 - val_acc: 0.2790
Epoch 4/20
6680/6680 [==============================] - 3s - loss: 10.0572 - acc: 0.3392 - val_loss:

6680/6680 [==============================] - 3s - loss: 9.4580 - acc: 0.3957 - val_loss: 9.8155 - val_acc: 0.3317
Epoch 8/20
6680/6680 [==============================] - 3s - loss: 9.4166 - acc: 0.4033 - val_loss: 9.8341 - val_acc: 0.3246
Epoch 9/20
6680/6680 [==============================] - 3s - loss: 9.3906 - acc: 0.4097 - val_loss: 9.8160 - val_acc: 0.3269
Epoch 10/20
6680/6680 [==============================] - 3s - loss: 9.2159 - acc: 0.4060 - val_loss: 9.5934 - val_acc: 0.3377
Epoch 11/20
6680/6680 [==============================] - 3s - loss: 9.0726 - acc: 0.4251 - val_loss: 9.6434 - val_acc: 0.3353
Epoch 12/20
6680/6680 [==============================] - 3s - loss: 8.9660 - acc: 0.4322 - val_loss: 9.3793 - val_acc: 0.3605
Epoch 13/20


6680/6680 [==============================] - 3s - loss: 8.7967 - acc: 0.4367 - val_loss: 9.2501 - val_acc: 0.3509
Epoch 14/20
6680/6680 [==============================] - 2s - loss: 8.5607 - acc: 0.4507 - val_loss: 9.0553 - val_acc: 0.3784
Epoch 15/20
6680/6680 [==============================] - 3s - loss: 8.4799 - acc: 0.4618 - val_loss: 9.0206 - val_acc: 0.3796
Epoch 16/20
6680/6680 [==============================] - 3s - loss: 8.3547 - acc: 0.4668 - val_loss: 8.8601 - val_acc: 0.3880
Epoch 17/20
6680/6680 [==============================] - 3s - loss: 8.1880 - acc: 0.4795 - val_loss: 8.8805 - val_acc: 0.3784
Epoch 18/20
6680/6680 [==============================] - 3s - loss: 8.1470 - acc: 0.4868 - val_loss: 8.7844 - val_acc: 0.3820
Epoch 19/20


6680/6680 [==============================] - 3s - loss: 8.1356 - acc: 0.4885 - val_loss: 8.8578 - val_acc: 0.3868
Epoch 20/20
6680/6680 [==============================] - 3s - loss: 8.0812 - acc: 0.4889 - val_loss: 8.6656 - val_acc: 0.3928
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 133)               272517    
Total params: 272,517.0
Trainable params: 272,517.0
Non-trainable params: 0.0
_________________________________________________________________
Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 4s - loss: 1.6494 - acc: 0.5918 - val_loss: 0.8382 - val_acc: 0.7485
Epoch 2/20
6680/6680 [==============================] - 3s - loss: 0.4435 - acc: 0.8639 - val_loss: 0.7485 - val_a

6680/6680 [==============================] - 4s - loss: 0.2690 - acc: 0.9142 - val_loss: 0.6607 - val_acc: 0.7976
Epoch 4/20
6680/6680 [==============================] - 4s - loss: 0.1821 - acc: 0.9410 - val_loss: 0.7072 - val_acc: 0.7952
Epoch 5/20
6680/6680 [==============================] - 4s - loss: 0.1240 - acc: 0.9615 - val_loss: 0.6658 - val_acc: 0.8192
Epoch 6/20
6680/6680 [==============================] - 4s - loss: 0.0897 - acc: 0.9734 - val_loss: 0.6728 - val_acc: 0.8156
Epoch 7/20


...

6680/6680 [==============================] - 9s - loss: 0.1280 - acc: 0.9638 - val_loss: 0.6099 - val_acc: 0.8539
Epoch 13/20
6680/6680 [==============================] - 9s - loss: 0.1185 - acc: 0.9629 - val_loss: 0.6032 - val_acc: 0.8467
Epoch 14/20


6680/6680 [==============================] - 9s - loss: 0.1088 - acc: 0.9674 - val_loss: 0.6280 - val_acc: 0.8611
Epoch 15/20
6680/6680 [==============================] - 9s - loss: 0.1001 - acc: 0.9684 - val_loss: 0.6567 - val_acc: 0.8467
Epoch 16/20


6680/6680 [==============================] - 9s - loss: 0.0944 - acc: 0.9720 - val_loss: 0.6636 - val_acc: 0.8479
Epoch 17/20
6680/6680 [==============================] - 9s - loss: 0.0889 - acc: 0.9749 - val_loss: 0.6564 - val_acc: 0.8479
Epoch 18/20


6680/6680 [==============================] - 9s - loss: 0.0841 - acc: 0.9769 - val_loss: 0.7012 - val_acc: 0.8575
Epoch 19/20
6680/6680 [==============================] - 9s - loss: 0.0770 - acc: 0.9781 - val_loss: 0.6545 - val_acc: 0.8575
Epoch 20/20


6680/6680 [==============================] - 9s - loss: 0.0723 - acc: 0.9792 - val_loss: 0.7066 - val_acc: 0.8515


In [29]:
print(results)

[(40.550239234449762, 'saved_models/weights.best.VGG19.hdf5'), (79.904306220095691, 'saved_models/weights.best.Resnet50.hdf5'), (80.502392344497608, 'saved_models/weights.best.InceptionV3.hdf5'), (85.167464114832541, 'saved_models/weights.best.Xception.hdf5')]


In [30]:
from keras.layers import Merge

def model1(network1, network2):
    ### Obtain Bottleneck Features

    bottleneck_features = np.load('bottleneck_features/Dog{}Data.npz'.format(network1))
    train_VGG16 = bottleneck_features['train']
    valid_VGG16 = bottleneck_features['valid']
    test_VGG16 = bottleneck_features['test']

    VGG16_model = Sequential()
    VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
    
    bottleneck_features = np.load('bottleneck_features/Dog{}Data.npz'.format(network2))
    train_network2 = bottleneck_features['train']
    valid_network2 = bottleneck_features['valid']
    test_network2 = bottleneck_features['test']

    model_2 = Sequential()
    model_2.add(GlobalAveragePooling2D(input_shape=train_network2.shape[1:]))
    
    merged = Merge([VGG16_model, model_2], mode='sum')
    
    merged_model = Sequential()
    merged_model.add(merged)
    merged_model.add(Dropout(0.7))
    merged_model.add(Dense(133, activation='softmax'))

    merged_model.summary()

    ### Compile the Model

    # merged_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    merged_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    ### Train the Model
    model_file = 'saved_models/weights.best.{}_{}.hdf5'.format(network1, network2)
    checkpointer = ModelCheckpoint(filepath=model_file, 
                                   verbose=1, save_best_only=True)

    merged_model.fit([train_VGG16, train_network2], train_targets, 
              validation_data=([valid_VGG16, valid_network2], valid_targets),
              epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

    ### Load the Model with the Best Validation Loss

    merged_model.load_weights(model_file)

    ### Test the Model

    # get index of predicted dog breed for each image in test set
    merged_predictions = [np.argmax(merged_model.predict([np.expand_dims(feature[0], axis=0), np.expand_dims(feature[1], axis=0)])) for feature in zip(test_VGG16, test_network2)]

    # report test accuracy
    test_accuracy = 100*np.sum(np.array(merged_predictions)==np.argmax(test_targets, axis=1))/len(merged_predictions)
    return (test_accuracy, VGG16_model)

In [31]:
merged_model = model1('Xception', 'Resnet50')

C:\Users\dep-nogi\Anaconda3\envs\aind-dog\lib\site-packages\ipykernel\__main__.py:22: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 2048)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 133)               272517    
Total params: 272,517.0
Trainable params: 272,517.0
Non-trainable params: 0.0
_________________________________________________________________
Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 10s - loss: 4.2072 - acc: 0.2024 - val_loss: 1.2374 - val_acc: 0.6934
Epoch 2/20
6680/6680 [==============================] - 8s - loss: 1.9554 - acc: 0.5019 - val_loss: 0.7746 - val_acc: 0.7784
Epoch 3/20


6680/6680 [==============================] - 8s - loss: 1.3678 - acc: 0.6225 - val_loss: 0.6380 - val_acc: 0.8060
Epoch 4/20
6680/6680 [==============================] - 8s - loss: 1.0888 - acc: 0.6885 - val_loss: 0.5672 - val_acc: 0.8275
Epoch 5/20


6680/6680 [==============================] - 9s - loss: 0.9221 - acc: 0.7278 - val_loss: 0.5376 - val_acc: 0.8287
Epoch 6/20
6680/6680 [==============================] - 8s - loss: 0.8299 - acc: 0.7537 - val_loss: 0.4829 - val_acc: 0.8467
Epoch 7/20


6680/6680 [==============================] - 8s - loss: 0.7562 - acc: 0.7737 - val_loss: 0.4695 - val_acc: 0.8539
Epoch 8/20
6680/6680 [==============================] - 8s - loss: 0.6919 - acc: 0.7898 - val_loss: 0.4816 - val_acc: 0.8527
Epoch 9/20


...

6680/6680 [==============================] - 8s - loss: 0.5638 - acc: 0.8274 - val_loss: 0.4329 - val_acc: 0.8611
Epoch 12/20
6680/6680 [==============================] - 9s - loss: 0.5140 - acc: 0.8395 - val_loss: 0.4406 - val_acc: 0.8563
Epoch 13/20


6680/6680 [==============================] - 9s - loss: 0.5019 - acc: 0.8454 - val_loss: 0.4144 - val_acc: 0.8635
Epoch 14/20
6680/6680 [==============================] - 8s - loss: 0.4667 - acc: 0.8516 - val_loss: 0.4173 - val_acc: 0.8635
Epoch 15/20


6680/6680 [==============================] - 8s - loss: 0.4238 - acc: 0.8654 - val_loss: 0.4102 - val_acc: 0.8623
Epoch 16/20
6680/6680 [==============================] - 8s - loss: 0.4238 - acc: 0.8672 - val_loss: 0.4045 - val_acc: 0.8695
Epoch 17/20


6680/6680 [==============================] - 8s - loss: 0.4098 - acc: 0.8677 - val_loss: 0.3892 - val_acc: 0.8814
Epoch 18/20
6680/6680 [==============================] - 8s - loss: 0.3955 - acc: 0.8760 - val_loss: 0.4056 - val_acc: 0.8754
Epoch 19/20


6680/6680 [==============================] - 8s - loss: 0.3878 - acc: 0.8728 - val_loss: 0.4010 - val_acc: 0.8802
Epoch 20/20
6680/6680 [==============================] - 8s - loss: 0.3629 - acc: 0.8814 - val_loss: 0.3939 - val_acc: 0.8814


In [33]:
print(merged_model)

(86.961722488038276, <keras.models.Sequential object at 0x0000021BA5487EF0>)


In [32]:
from keras.utils.vis_utils import plot_model
plot_model(merged_model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [157]:
from keras.layers import Merge

def model1(network1):
    ### Obtain Bottleneck Features

    bottleneck_features = np.load('bottleneck_features/Dog{}Data.npz'.format(network1))
    train_VGG16 = bottleneck_features['train']
    valid_VGG16 = bottleneck_features['valid']
    test_VGG16 = bottleneck_features['test']

    model = Sequential()
    model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
    model.add(Dropout(0.7))
    model.add(Dense(133, activation='softmax'))

    model.summary()

    ### Compile the Model

    # merged_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    ### Train the Model
    model_file = 'saved_models/weights.best.{}.hdf5'.format(network1)
    checkpointer = ModelCheckpoint(filepath=model_file, 
                                   verbose=1, save_best_only=True)

    model.fit(train_VGG16, train_targets, 
              validation_data=(valid_VGG16, valid_targets),
              epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

    ### Load the Model with the Best Validation Loss

    model.load_weights(model_file)

    ### Test the Model

    # get index of predicted dog breed for each image in test set
    merged_predictions = [np.argmax(model.predict(np.expand_dims(feature[0], axis=0), np.expand_dims(feature, axis=0))) for feature in test_VGG16]

    # report test accuracy
    test_accuracy = 100*np.sum(np.array(merged_predictions)==np.argmax(test_targets, axis=1))/len(merged_predictions)
    return (test_accuracy, model)

In [ ]:
model_xception = model1("Xception")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_13  (None, 2048)              0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 133)               272517    
Total params: 272,517.0
Trainable params: 272,517.0
Non-trainable params: 0.0
_________________________________________________________________
Train on 6680 samples, validate on 835 samples
Epoch 1/20


In [ ]:
print(model_xception)

In [ ]:
### TODO: Obtain bottleneck features from another pre-trained CNN.

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  At the end of your code cell block, summarize the layers of your model by executing the line:
    
        <your model's name>.summary()
   
__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 



In [ ]:
### TODO: Define your architecture.

### (IMPLEMENTATION) Compile the Model

In [ ]:
### TODO: Compile the model.

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.  

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [ ]:
### TODO: Train the model.

### (IMPLEMENTATION) Load the Model with the Best Validation Loss

In [ ]:
### TODO: Load the model weights with the best validation loss.

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Ensure that your test accuracy is greater than 60%.

In [ ]:
### TODO: Calculate classification accuracy on the test dataset.

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan_hound`, etc) that is predicted by your model.  

Similar to the analogous function in Step 5, your function should have three steps:
1. Extract the bottleneck features corresponding to the chosen CNN model.
2. Supply the bottleneck features as input to the model to return the predicted vector.  Note that the argmax of this prediction vector gives the index of the predicted dog breed.
3. Use the `dog_names` array defined in Step 0 of this notebook to return the corresponding breed.

The functions to extract the bottleneck features can be found in `extract_bottleneck_features.py`, and they have been imported in an earlier code cell.  To obtain the bottleneck features corresponding to your chosen CNN architecture, you need to use the function

    extract_{network}
    
where `{network}`, in the above filename, should be one of `VGG19`, `Resnet50`, `InceptionV3`, or `Xception`.

In [150]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.
def predict(image_path):
    bottleneck_feature = extract_Xception(path_to_tensor(image_path))
    #                    extract_Resnet50(path_to_tensor(image_path))]
    predicted_prob = merged_model[-1].predict(bottleneck_feature)
    print(predicted_prob.shape)
    print(predicted_prob)
    predicted_idx = np.argmax(predicted_prob[0])
    print(predicted_idx)
    return dog_names[predicted_idx]

In [78]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

---
<a id='step7'></a>
## Step 7: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that __you__ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ 

In [84]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.
my_image_paths = ["test_images/dog4.jpg",
                "test_images/philippe_jordan.jpg", 
                  "test_images/eric_clapton.jpg",
                  "test_images/shibainu.jpg", 
                  "test_images/pag.jpg", 
                 "test_images/dog2.jpg",
                 ]

In [151]:
for image_path in my_image_paths:
    print(predict(image_path))

(1, 2048)
[[ 0.34740329  0.          0.         ...,  0.00185373  0.          0.        ]]
160


IndexError: list index out of range

In [57]:
print(dog_names)

['Affenpinscher', 'Afghan_hound', 'Airedale_terrier', 'Akita', 'Alaskan_malamute', 'American_eskimo_dog', 'American_foxhound', 'American_staffordshire_terrier', 'American_water_spaniel', 'Anatolian_shepherd_dog', 'Australian_cattle_dog', 'Australian_shepherd', 'Australian_terrier', 'Basenji', 'Basset_hound', 'Beagle', 'Bearded_collie', 'Beauceron', 'Bedlington_terrier', 'Belgian_malinois', 'Belgian_sheepdog', 'Belgian_tervuren', 'Bernese_mountain_dog', 'Bichon_frise', 'Black_and_tan_coonhound', 'Black_russian_terrier', 'Bloodhound', 'Bluetick_coonhound', 'Border_collie', 'Border_terrier', 'Borzoi', 'Boston_terrier', 'Bouvier_des_flandres', 'Boxer', 'Boykin_spaniel', 'Briard', 'Brittany', 'Brussels_griffon', 'Bull_terrier', 'Bulldog', 'Bullmastiff', 'Cairn_terrier', 'Canaan_dog', 'Cane_corso', 'Cardigan_welsh_corgi', 'Cavalier_king_charles_spaniel', 'Chesapeake_bay_retriever', 'Chihuahua', 'Chinese_crested', 'Chinese_shar-pei', 'Chow_chow', 'Clumber_spaniel', 'Cocker_spaniel', 'Collie',

In [133]:
len(dog_names)

133